# CONVERT2ICEBERG Pickler

Once Convert2Iceberg updates are completed, the source code and files can be pickled using this notebook. This notebook is meant to be run from this location in the repo to package up the Convert2Iceberg files into lists that are picked and then written to a file. 

These files will be used by the Convert2Iceberg Installer to install Convert2Iceberg.


In [103]:
import base64
import codecs
import os
import pickle

## STEP 1: Function definition

In [104]:
def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)

def encode(obj):
    pickled = pickle.dumps(obj)
    return codecs.encode(pickled, 'base64').decode()   

def get_files(path):
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f != ".DS_Store"]
    files.sort()
    for file in files:
        yield file

## STEP 2: Set directory paths

In [105]:
#get this repo's root dir
repo_root = os.path.dirname(os.getcwd())

#set pickle directory
pickle_dir = f"{repo_root}/pickles"

#set scripts/imgs directories
c2i_setup_dir = f"{repo_root}/convert2iceberg/scripts"
c2i_streamlit_main = f"{repo_root}/convert2iceberg/streamlit/main.py"
c2i_streamlit_env = f"{repo_root}/convert2iceberg/streamlit/environment.yml"
c2i_streamlit_utils_dir = f"{repo_root}/convert2iceberg/streamlit/utils"
c2i_streamlit_img_dir = f"{repo_root}/convert2iceberg/streamlit/img"

## STEP 3: Add content of directory files to list

In [106]:
#c2i_setup
c2i_setup_setup_dict = {}
for file in get_files(c2i_setup_dir):
    with open(c2i_setup_dir + "/" + file, "rb") as current_file:
        c2i_setup_setup_dict[file] = current_file.read()
        
#c2i_streamlit_main
c2i_streamlit_main_dict = {}
with open(c2i_streamlit_main, "rb") as current_file:
    c2i_streamlit_main_dict["main.py"] = current_file.read()
    
#c2i_streamlit_env
c2i_streamlit_env_dict = {}
with open(c2i_streamlit_env, "rb") as current_file:
    c2i_streamlit_env_dict["environment.yml"] = current_file.read()
        
#c2i_streamlit_utils
c2i_streamlit_utils_dict = {}
for file in get_files(c2i_streamlit_utils_dir):
    with open(c2i_streamlit_utils_dir + "/" + file, "rb") as current_file:
        c2i_streamlit_utils_dict[file] = current_file.read()

#c2i_streamlit_img
c2i_streamlit_img_dict = {}
for file in get_files(c2i_streamlit_img_dir):
    with open(c2i_streamlit_img_dir + "/" + file, "rb") as current_file:
        encoded_img = base64.b64encode(current_file.read())
        c2i_streamlit_img_dict[file] = encoded_img

## STEP 4: Create pickle files

In [107]:
#write c2i setup pickle file
c2i_obj = {
    "c2i_setup" : c2i_setup_setup_dict,
    "c2i_streamlit_main" : c2i_streamlit_main_dict,
    "c2i_streamlit_env" : c2i_streamlit_env_dict,
    "c2i_streamlit_utils" : c2i_streamlit_utils_dict,
    "c2i_streamlit_imgs" : c2i_streamlit_img_dict,
}
encoded_c2i_objj = encode(c2i_obj)
with open(f"{pickle_dir}/c2i_setup.pickle", "w") as p:
    p.write(encoded_c2i_objj)     

## STEP 5: Decode pickles to verify contents (optional)
**Recommendation:** Copy each decoded string to a text editor for better readability.

In [108]:
print(decode(encoded_c2i_objj))

{'c2i_setup': {'01_account_setup.sql': b'SET C2I_ROLE = \'<MY_ROLE>\';\nSET C2I_DB = \'CONVERT2ICEBERG_<MY_ROLE>\';\nSET C2I_WH = \'CONVERT2ICEBERG_<MY_ROLE>_WH\';\n\nUSE ROLE IDENTIFIER($C2I_ROLE);\nCREATE WAREHOUSE IF NOT EXISTS IDENTIFIER($C2I_WH) WITH WAREHOUSE_SIZE = \'XSMALL\';\n\n--create CONVERT2ICEBERG DB\nUSE WAREHOUSE IDENTIFIER($C2I_WH);\nCREATE OR REPLACE DATABASE IDENTIFIER($C2I_DB);\n\nUSE DATABASE IDENTIFIER($C2I_DB);\nCREATE OR REPLACE SCHEMA LOGGING;\nCREATE OR REPLACE SCHEMA STREAMLIT;\n\n--create CONVERSION_LOG table\nUSE SCHEMA LOGGING;\nCREATE TABLE CONVERSION_LOG(\n    id VARCHAR\n    ,conversion_type VARCHAR\n    ,original_tables ARRAY --one table if conversion_type is table, one or more if conversion_type is query\n    ,original_query VARCHAR --null if conversion_type is table\n    ,original_query_id VARCHAR --null if conversion_type is table\n    ,converted_tables ARRAY --one table if conversion_type is table, one or more if conversion_type is query\n    ,conv